In [4]:
#Needed in general
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py as h5
import gc

#Quick fudge to make import from ../Scripts work
sys.path.append('../../Scripts')

#Custom imports
import ClassCOMPAS     as CC ###
# import ClassFormationChannels as FC a


# import script that has formation channel classification functions:
from PostProcessingScripts import * 
# from ClassFormationChannels_5mainchannels import * 



from astropy import units as u
from astropy import constants as const
from astropy import constants as c

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
print(fs)

import astropy
import seaborn as sns
# from seaborn_utility import cmap_from_colour

24


In [9]:
M1 = 1.35 #* c.M_sun 
M2 = 3* M1 #
BH_chi = np.linspace(-1, 0.97, 11)
NS_radii= 11 #km 

Mej = calculateEjectedMassMerger(m_ns=np.asarray([M2]*11), r_ns=np.asarray([NS_radii]*11), m_bh=np.asarray([M1]*11), Xeff=BH_chi)
print(BH_chi)
print(Mej)


[-1.    -0.803 -0.606 -0.409 -0.212 -0.015  0.182  0.379  0.576  0.773
  0.97 ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
def print_disrupted_BHNS(BPSmodelName='P', mssfr = '112'):
    """
    
    mssfr = '112' # mssfr that matches LIGO observed rates.
    """
    
    
    DCOtype='BHNS'
    # path for files 
    path_ = '/Volumes/Andromeda/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
    path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'

    # read in data 
    fdata = h5.File(path)


    # M1 will be the most massive, M2 the least massive compact object. 
    M1, M2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
    Chirpmass = chirpmass(M1, M2)
    MassRatio = M2/M1

    ### read in MSSFR weights: ###
    
    # get detected weights
    fparam_key = 'weights_detected'
    weightheader = 'w_' + mssfr
    w = fdata[fparam_key][weightheader][...].squeeze()

    labellist=[ r'$\rm{m}_{\rm{chirp}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{BH}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{NS}} \ [\rm{M}_{\odot}]$',\
           r'$q $'] #,  r'$\log_{10}(t) $']              
    varlist = [Chirpmass, M1, M2, MassRatio]


    levels = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
    level_labels = [r"$98\%$", r"$90\%$", r"$75\%$", r"$50\%$", r"$25\%$", r"$10\%$", r"$0\%$"]



    maskGW200105  = ((M1 <= (8.9+1.2))  & (M1>=(8.9-1.5))) & ((M2 <= (1.9+0.3))  & (M2>=(1.9-0.2))) & ((chirpmass(M1,M2)>=(3.41-0.07)) & (chirpmass(M1,M2)<=(3.41+0.08))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35)))
    maskGW200115  = ((M1 <= (5.7+1.8))  & (M1>=(5.7-2.1))) & ((M2 <= (1.5+0.7))  & (M2>=(1.5-0.3))) & ((chirpmass(M1,M2)>=(2.42-0.07)) & (chirpmass(M1,M2)<=(2.42+0.05))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35))) 

    
    NS_radii = 13.0 # km 
    BH_chi = 0.97 
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    
    
    print()
    NS_radii = 13.0 # km 
    BH_chi = 0
    print('BH spin =',BH_chi)
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    
    
    print()
    NS_radii = 13.0 # km 
    BH_chi = 0.08 + 0.22 
    print('BH spin =',BH_chi)
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    print()
    NS_radii = 13.0 # km 
    BH_chi = 0.33+0.48
    print('BH spin =',BH_chi)
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    print('--------')
    print()
    NS_radii = 11.0 # km 
    BH_chi = 0.08 + 0.22 
    print('BH spin =',BH_chi)
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    print()
    NS_radii = 11.0 # km 
    BH_chi = 0.33+0.48
    print('BH spin =',BH_chi)
    Mej = calculateEjectedMassMerger(m_ns=M2, r_ns=NS_radii, m_bh=M1, Xeff=BH_chi)
    mask_disrupted = Mej > (1E-6) # Msun 
    
    print('fraction of all BHNS = ', np.sum(w[mask_disrupted])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_disrupted==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_disrupted==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )    

    return 




print_disrupted_BHNS()



fraction of all BHNS =  0.9989974929972938
fraction of all GW200115 =  1.0
fraction of all GW200105 =  1.0

BH spin = 0
fraction of all BHNS =  0.07590750641956946
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

BH spin = 0.3
fraction of all BHNS =  0.19456673369212743
fraction of all GW200115 =  0.896277098362517
fraction of all GW200105 =  0.0

BH spin = 0.81
fraction of all BHNS =  0.7554881289772046
fraction of all GW200115 =  1.0
fraction of all GW200105 =  1.0
--------

BH spin = 0.3
fraction of all BHNS =  0.061382772438404704
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

BH spin = 0.81
fraction of all BHNS =  0.5054313535090036
fraction of all GW200115 =  1.0
fraction of all GW200105 =  0.0
